In [1]:
# импорт необходимых библиотек
import pandas as pd
import os
from docx import Document

In [2]:
# путь к файлам
PACH_DATA = "../raw_data/"

In [3]:
# вывод файлов директории "data"
data_list = os.listdir(PACH_DATA)
data_list

['1_Многоквартирные дома с технико-экономическими характеристиками.xlsx',
 '2_Инциденты,_зарегистрированные_на_объектах_городского_хозяйства (2).xlsx',
 '2_Инциденты,_зарегистрированные_на_объектах_городского_хозяйства.xlsx',
 '3_Работы по капитальному ремонту, проведенные в многоквартирных домах.xlsx',
 '4_Виды работ по капитальному ремонту многоквартирных домов.xlsx',
 '4_Виды работ по содержанию и ремонту общего имущества многоквартирных домов.xlsx',
 '4_Виды_работ_по_капитальному_ремонту_многоквартирных_домов_с_фактом (3).xlsx',
 '5_Типы событий, регистрируемых по типу объекта многоквартирный дом.xlsx',
 'Объекты_модели_Группы_видов_работ_по_содержанию_и_ремонту_общего.xlsx',
 'Описание примеров данных.docx']

### Описание примеров данных

In [4]:
# загрузка данных файла 'Описание примеров данных.docx'
document = Document(PACH_DATA + data_list[-1])

In [5]:
# парсинг таблиц данных файла 'Описание примеров данных.docx'
df_tables = []
for table in document.tables:
    df = [['' for i in range(len(table.columns))] for j in range(len(table.rows))]
    for i, row in enumerate(table.rows):
        for j, cell in enumerate(row.cells):
            if cell.text:
                df[i][j] = cell.text
    df_tables.append(pd.DataFrame(df[1:], columns=df[0]))

In [6]:
# количество обнаруженных таблиц
len(df_tables)

16

### Типы событий, регистрируемых по типу объекта многоквартирный дом

In [7]:
# путь к данным
data_list[7]

'5_Типы событий, регистрируемых по типу объекта многоквартирный дом.xlsx'

In [8]:
# описание данных
df_tables[15]

,Имя колонки,Комментарий
0,id,Идентификатор
1,name,Наименование типа события
2,system,Информационная система – источник события


In [9]:
# вывод наименование листов в файле "5_Типы событий, регистрируемых по типу объекта многоквартирный дом.xlsx"
xlsx_file = pd.ExcelFile(PACH_DATA + data_list[7])
sheet_names = xlsx_file.sheet_names
print(sheet_names)

['Лист1']


In [10]:
# загрузка данных
events_df = pd.read_excel(PACH_DATA + data_list[7], sheet_name="Лист1")
events_df.head()

,id,name,system
0,001b91e9-2ac5-4949-bff0-bb51d4f3057c,Нарушение в работе АГВ,MOS_GAS
1,0089fd48-aa7a-443d-85a5-00470285b076,Проверить на загазованность дюкер Гольяновский,MOS_GAS
2,016697a2-feef-4568-9629-3c9dddae3aea,Разрушение асфальтового полотна вокруг ковера,MOS_GAS
3,02212d7e-3c4c-4e6a-8f6b-d4f8f1b82e72,Угроза взрыва; Запах газа в доме,MOS_GAS
4,0250d455-608a-4d63-9902-ebe3b82d0b67,Запах газа от ГРП,MOS_GAS


In [11]:
# проверка на наличие пропусков
events_df.isna().sum()

id        0
name      1
system    0
dtype: int64

In [12]:
# удаление пропусков
events_df = events_df.dropna()

In [13]:
#  проверка на наличие дубликатов
events_df.iloc[:,1:].duplicated().sum()

46

In [14]:
# удаление признака "id"
events_df = events_df.drop(columns="id")

In [15]:
# размерность данных
events_df.shape

(949, 2)

In [16]:
# удаление дубликатов
events_df = events_df.drop_duplicates()

In [17]:
# размерность данных
events_df.shape

(903, 2)

In [18]:
# наименование систем
system_names = events_df["system"].unique()
system_names

array(['MOS_GAS', 'MVK', 'EDC', 'ASUPR', 'MGI', 'NG', 'EVAGD', 'KGH',
       'GORMOST', 'CAFAP'], dtype=object)

In [38]:
# функция извлечения уникальных наименований событий по системам
def get_name(col_name):
    names = events_df.query(f"system  == '{col_name}'")["name"].values
    data = []
    for name in names:
        if len(name.split(";")) == 1:
           if name not in data:
               data.append(name)
        else:
            for nam in name.split(";"):
                if nam not in data:
                    data.append(nam)
    return {"incident_type" : data}

In [39]:
# извлечение уникальных наименований событий по системам
data = []
for col_name in system_names:
    df = pd.DataFrame(get_name(col_name))
    df["system"] = col_name
    data.append(df)

In [41]:
# объединение данных
events_df = pd.concat(data)
events_df.head()

,incident_type,system
0,Нарушение в работе АГВ,MOS_GAS
1,Проверить на загазованность дюкер Гольяновский,MOS_GAS
2,Разрушение асфальтового полотна вокруг ковера,MOS_GAS
3,Угроза взрыва,MOS_GAS
4,Запах газа в доме,MOS_GAS


In [42]:
# запись данных в файл
events_df.to_csv("../data/events_df.csv", index=False)